In [26]:

import tensorflow as tf
import numpy as np
from pprint import pprint

saveRoot = "/atlas/u/esheehan/wikipedia_project/dataset/image_dataset/CS325B/"
dataRootPath = "/atlas/u/chenlin/wikipedia_images/Poverty_prediction/"
toConvert = [
#             'lx_median_2009-11_angola_dhslocs_ee_export.tfrecord.gz', 
#              'lx_median_2009-11_burkina_faso_dhslocs_ee_export.tfrecord.gz', 
#              'lx_median_2009-11_cameroon_dhslocs_ee_export.tfrecord.gz', 
#              'lx_median_2009-11_ethiopia_dhslocs_ee_export.tfrecord.gz', 
#              'lx_median_2009-11_lesotho_dhslocs_ee_export.tfrecord.gz', 
#              'lx_median_2009-11_malawi_dhslocs_ee_export.tfrecord.gz', 
#              'lx_median_2009-11_mozambique_dhslocs_ee_export.tfrecord.gz', 
#              'lx_median_2009-11_nigeria_dhslocs_ee_export.tfrecord.gz', 
#              'lx_median_2009-11_rwanda_dhslocs_ee_export.tfrecord.gz', 
#              'lx_median_2009-11_senegal_dhslocs_ee_export.tfrecord.gz', 
#              'lx_median_2009-11_tanzania_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2009-11_uganda_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2009-11_zimbabwe_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_benin_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_burkina_faso_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_cote_d_ivoire_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_drc_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_ghana_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_guinea_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_kenya_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_lesotho_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_malawi_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_mali_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_nigeria_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_rwanda_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_senegal_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_sierra_leone_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_togo_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_uganda_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2012-14_zambia_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2015-17_angola_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2015-17_ethiopia_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2015-17_ghana_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2015-17_kenya_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2015-17_malawi_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2015-17_mali_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2015-17_nigeria_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2015-17_tanzania_dhslocs_ee_export.tfrecord.gz', 
             'lx_median_2015-17_zimbabwe_dhslocs_ee_export.tfrecord.gz'
            ]

options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
iterator = tf.python_io.tf_record_iterator(TFRecords_file, options=options)


In [27]:
# get the first Example stored in the TFRecords file
record_str = next(iterator)

# parse the binary string
ex = tf.train.Example.FromString(record_str)  # parse into an actual Example message
features = ex.features  # get the Features message within the Example
feature_map = features.feature  # get the mapping from feature name strings to Feature

# use the WhichOneof() method on messages with `oneof` fields to
# determine the type of the field
feature_types = {}
for name in feature_map.keys():
    feature_types[name] = feature_map[name].WhichOneof('kind')

print('Features and types:')
pprint(feature_types)

Features and types:
{'ASP': 'float_list',
 'BLUE': 'float_list',
 'ELEV': 'float_list',
 'GREEN': 'float_list',
 'LAT': 'float_list',
 'LON': 'float_list',
 'NIGHTLIGHTS': 'float_list',
 'NIR': 'float_list',
 'RED': 'float_list',
 'SLO': 'float_list',
 'SWIR1': 'float_list',
 'SWIR2': 'float_list',
 'TEMP1': 'float_list',
 'cluster_index': 'float_list',
 'country': 'bytes_list',
 'households': 'float_list',
 'indx': 'float_list',
 'lon': 'float_list',
 'system:index': 'bytes_list',
 'urban_rural': 'float_list',
 'wealth': 'float_list',
 'wealthpooled': 'float_list',
 'wealthpooled5country': 'float_list',
 'year': 'float_list'}


In [28]:
def makeCSV(TFRecords_file, saveFile):
    print("Evaluating file: ", TFRecords_file)
    options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
    iterator = tf.python_io.tf_record_iterator(TFRecords_file, options=options)

    reqBands = ['LAT', 'LON', 'wealthpooled']

    result = [] # (lat,lon), wealthpooled 

    for record_str in iterator:
        ex = tf.train.Example.FromString(record_str)  # parse into an actual Example message
        feature_map = ex.features.feature  # get the mapping from feature name strings to Feature

        curLat = np.mean(feature_map['LAT'].float_list.value)
        curLon = np.mean(feature_map['LON'].float_list.value)
        curWealth = feature_map['wealthpooled'].float_list.value[0]

        result.append(((curLat, curLon), curWealth))

    print(len(result))
    print(result[0])
     
    print("Saving file to ", saveFile)
    np.save(saveFile, result)

In [29]:
for file in toConvert:
    makeCSV(dataRootPath + file, saveRoot + file + ".npy")

Evaluating file:  /atlas/u/chenlin/wikipedia_images/Poverty_prediction/lx_median_2009-11_uganda_dhslocs_ee_export.tfrecord.gz
570
((0.6894852133358226, 30.081389849793677), -0.5883581638336182)
Saving file to  /atlas/u/esheehan/wikipedia_project/dataset/image_dataset/CS325B/lx_median_2009-11_uganda_dhslocs_ee_export.tfrecord.gz.npy
Evaluating file:  /atlas/u/chenlin/wikipedia_images/Poverty_prediction/lx_median_2009-11_zimbabwe_dhslocs_ee_export.tfrecord.gz
393
((-16.370638095631318, 31.80669418035769), -0.4672582745552063)
Saving file to  /atlas/u/esheehan/wikipedia_project/dataset/image_dataset/CS325B/lx_median_2009-11_zimbabwe_dhslocs_ee_export.tfrecord.gz.npy
Evaluating file:  /atlas/u/chenlin/wikipedia_images/Poverty_prediction/lx_median_2012-14_benin_dhslocs_ee_export.tfrecord.gz
746
((6.347452288982915, 2.4010620079788505), 1.3024872541427612)
Saving file to  /atlas/u/esheehan/wikipedia_project/dataset/image_dataset/CS325B/lx_median_2012-14_benin_dhslocs_ee_export.tfrecord.gz.np

322
((7.961401348488003, 3.5903416119369806), 1.7204735279083252)
Saving file to  /atlas/u/esheehan/wikipedia_project/dataset/image_dataset/CS325B/lx_median_2015-17_nigeria_dhslocs_ee_export.tfrecord.gz.npy
Evaluating file:  /atlas/u/chenlin/wikipedia_images/Poverty_prediction/lx_median_2015-17_tanzania_dhslocs_ee_export.tfrecord.gz
608
((-1.3816994578230615, 34.576290025898054), -0.5321860313415527)
Saving file to  /atlas/u/esheehan/wikipedia_project/dataset/image_dataset/CS325B/lx_median_2015-17_tanzania_dhslocs_ee_export.tfrecord.gz.npy
Evaluating file:  /atlas/u/chenlin/wikipedia_images/Poverty_prediction/lx_median_2015-17_zimbabwe_dhslocs_ee_export.tfrecord.gz
400
((-16.227597247852998, 31.72072540731991), -0.49163389205932617)
Saving file to  /atlas/u/esheehan/wikipedia_project/dataset/image_dataset/CS325B/lx_median_2015-17_zimbabwe_dhslocs_ee_export.tfrecord.gz.npy
